In [1]:
from whatwhy.text_processing.helper_methods import get_df_from_file, get_csv_string_from_df
from whatwhy.data_analysis.helper_methods import get_text_as_list
import pandas as pd
import numpy as np
csv_file_name = "/home/stevengt/Documents/code/whatwhy-data/News-Articles/all-the-news/wh_phrases.csv"

In [2]:
df = get_df_from_file(csv_file_name)
df["what tokens"] = df["what tokens"].apply(get_text_as_list)
df["why tokens"] = df["why tokens"].apply(get_text_as_list)
df = df[pd.notnull(df["what tokens"])]
df = df[pd.notnull(df["why tokens"])]

In [3]:
def get_vocab_counts(df, col_name):
    counts = {}
    for tokens in df[col_name]:
        for token in tokens:
            count = counts.get(token)
            if count is None:
                count = 0
            counts[token] = count + 1
    return counts

In [4]:
def remove_uncommon_whatwhy_tokens(df):
    what_counts = get_vocab_counts(df, "what tokens")
    why_counts = get_vocab_counts(df, "why tokens")
    df["what tokens"] = df["what tokens"].apply(lambda x: [token for token in x if what_counts[token] >=30 ])
    df["why tokens"] = df["why tokens"].apply(lambda x: [token for token in x if why_counts[token] >=30 ])
    return df

In [5]:
df = remove_uncommon_whatwhy_tokens(df)

In [6]:
csv_string = get_csv_string_from_df(df)
with open("/home/stevengt/Documents/code/whatwhy-data/News-Articles/all-the-news/wh_phrases2.csv", "w") as outfile:
    outfile.write(csv_string)